In [7]:
from sklearn.linear_model import LogisticRegression
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.learning_curve import validation_curve
import datetime

In [8]:
train_path = os.path.join('..','data','train_data_v3_processed.csv')
train_data = pd.read_csv(train_path)

In [9]:
df = train_data#[:10000]

In [10]:
df.head()

,id,question1,question2,is_duplicate,question1_lemma,question1_tag,question2_lemma,question2_tag,fuzz_ratio,fuzz_partial_ratio,...,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words,avg_word_q1,avg_word_q2
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what be the step by step guide to invest in sh...,WP VBZ DT NN IN NN NN TO VB IN NN NN IN NN .,what be the step by step guide to invest in sh...,WP VBZ DT NN IN NN NN TO VB IN NN NN .,93,98,...,66,57,9,20,20,14,12,12,3.785714,3.833333
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,what be the story of kohinoor ( koh - i - noor...,WP VBZ DT NN IN NNP -LRB- NNP HYPH NNP HYPH NN...,what would happen if the indian government ste...,WP MD VB IN DT JJ NN VBD DT NNP -LRB- NNP HYPH...,68,78,...,51,88,-37,21,29,8,13,11,5.500000,5.846154
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,how can -PRON- increase the speed of -PRON- in...,WRB MD PRP VB DT NN IN PRP$ NN NN IN VBG DT NNP .,how can internet speed be increase by hack thr...,WRB MD VB NN VB VBN IN VBG IN NN .,35,44,...,73,59,14,25,24,14,10,6,4.285714,5.000000
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,why be -PRON- mentally very lonely ? how can -...,WRB VBP PRP RB RB JJ . WRB MD PRP VB PRP .,find the remainder when [ math]23^{24}[/math ]...,VB DT NN WRB -LRB- NN -RRB- VBZ VBN IN CD .,17,22,...,50,65,-15,19,26,11,9,2,3.636364,6.333333
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"which one dissolve in water quikly sugar , sal...","WDT CD NN IN NN RB NN , NN , NN CC NN FW NN .",which fish would survive in salt water ?,WDT NN MD VB IN NN NN .,37,55,...,76,39,37,25,18,13,7,5,4.923077,4.714286


In [11]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60.
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [13]:
x_columns = ['fuzz_ratio',
       'fuzz_partial_ratio', 'fuzz_partial_token_set_ratio',
       'fuzz_partial_token_sort_ratio', 'fuzz_token_set_ratio',
       'fuzz_token_sort_ratio', 'len_q1', 'len_q2', 'diff_len', 'len_char_q1',
       'len_char_q2', 'len_word_q1', 'len_word_q2', 'common_words',
       'avg_word_q1', 'avg_word_q2']

y_columns = ['is_duplicate']

data_y = df[y_columns].values
data_x = df[x_columns].values
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.20, random_state=42)
#train = train.T
#test = test.T

In [22]:
train_global_df = pd.DataFrame(np.concatenate((train_x, train_y), axis=1))
train_global_df.columns = x_columns + y_columns

data_y = train_global_df[y_columns].values
data_x = train_global_df[x_columns].values

train_x, validation_x, train_y, validation_y = train_test_split(data_x, data_y, test_size=0.20, random_state=42)

train_y = train_y.astype('int')
validation_y = validation_y.astype('int')

In [23]:
C_param_range = [0.001,0.01,0.1,1,10,100,1000]

accuracy_table = pd.DataFrame(columns = ['C_parameter','Training Accuracy', 'Testing Accuracy'])
accuracy_table['C_parameter'] = C_param_range

j = 0
for i in C_param_range:
    start_time = datetime.datetime.now()
    
    lr = LogisticRegression(penalty = 'l2', C = i,random_state = 42)
    lr.fit(train_x,train_y)
    validation_y_pred = lr.predict(validation_x)
    
    accuracy_table.iloc[j,1] = accuracy_score(validation_y,validation_y_pred)
    
    
    test_y_pred = lr.predict(test_x)
    accuracy_table.iloc[j,2] = accuracy_score(test_y,test_y_pred)
    j += 1
    
    end_time = datetime.datetime.now()
    seconds_elapsed = (end_time - start_time).total_seconds()
    print("C = ",i,"It took {} to execute this".format(hms_string(seconds_elapsed)))
    
accuracy_table['Diff'] = np.abs(accuracy_table.iloc[:, 1]-accuracy_table.iloc[:, 2])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


C =  0.001 It took 0:00:22.60 to execute this
C =  0.01 It took 0:00:36.82 to execute this
C =  0.1 It took 0:00:51.00 to execute this
C =  1 It took 0:00:50.31 to execute this
C =  10 It took 0:00:46.67 to execute this
C =  100 It took 0:00:59.57 to execute this
C =  1000 It took 0:01:08.91 to execute this


In [33]:
accuracy_table.head()

,C_parameter,Training Accuracy,Testing Accuracy,Diff
0,0.001,0.660453,0.662919,0.00246594
1,0.010,0.662099,0.665118,0.00301955
2,0.100,0.663725,0.666481,0.00275632
3,1.000,0.665215,0.667967,0.00275247
4,10.000,0.665312,0.667875,0.00256278
